# 서울 지하철 2·4·5호선 이벤트 발생과 혼잡도 연관성 분석

## 1. 분석 개요

* **배경**: 대규모 이벤트(스포츠, 행사, 시위) 발생 시 지하철 이동 패턴 변화 및 혼잡도 급증 대응 필요
* **목적**: 이벤트 발생과 지하철 인원 간 상관관계 분석을 통한 "이벤트가 혼잡도에 유의미한 영향을 미치는가?" 가설 검증
* **가치**: 혼잡 유발 요인 사전 파악을 통한 안전사고 예방 및 효율적인 운행/인력 배치 전략 수립 근거 마련

## 2. 분석 대상 및 데이터 정의

### 2.1 데이터 출처 및 수집 방법

* **지하철 데이터**: 서울시 공공데이터 포털 (2·4·5호선 승하차 인원 합계)
* **이벤트 데이터**:
  * 행사: KOPIS 공연예술통합전산망 API
  * 야구: KBO 공식 홈페이지 크롤링
  * 시위: 빅카인즈(Big Kinds) 보도 일정 크롤링, 서울교통공사 지하철알림정보-공공데이터포털 API

### 2.2 시설 선정 및 정제 기준

분석 정밀도 향상을 위해 좌석 수 2,000석 이상의 중·대형 시설을 선별함

* **주요 분석 시설**:
  * 잠실종합운동장: 주경기장(64,623석), 야구장(24,411석), 실내체육관(11,032석)
  * 올림픽공원: KSPO DOME(15,000석), 핸드볼경기장(5,003석), 올림픽홀(2,952석)
* **제외 시설**: 올림픽공원 우리금융아트홀(1,184석) 등 2,000석 미만 시설 (데이터 노이즈 제거 목적)

### 2.3 분석 범위

* **기간**: 2023년 1월 ~ 2025년 12월
* **최종 표본**: 다중 이벤트 통합 적용 후 총 737건의 유효 사례 도출

## 3. 분석 로직 및 계산 방법론

### 3.1 데이터 전처리 및 매칭

* **역명 정규화**: 데이터셋 간 역명 불일치 해소 (예: "종합운동장역" → "종합운동장")
* **시공간 매칭**: [날짜, 호선, 역명]을 Key값으로 지하철-이벤트 데이터 결합

### 3.2 다중 이벤트 통합 (Grouping)

* **방법**: GROUP BY [날짜, 호선, 역명] 적용 및 카테고리 병합(join)
* **결과**: 총 758건 → 737건으로 정제 (동일 날짜 야구+콘서트 시 "야구, 행사"로 통합)

### 3.3 베이스라인(Baseline) 및 상승률 계산

* **Baseline**: 이벤트 미발생일의 동일 요일 평균 인원 ($[호선+역+요일]$ 기준)
* **상승률 공식**:

$$상승률(\%) = \frac{\text{이벤트 당일 승하차 합계} - \text{Baseline}}{\text{Baseline}} \times 100$$

## 4. 분석 결과

### 환경 세팅 및 라이브러리 로드

In [1]:
# 1. 환경 세팅 (설치)
!pip install seaborn

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import matplotlib.font_manager as fm

# 윈도우 한글 폰트 설정 (전역 적용)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

print("폰트 설정 완료!")

폰트 설정 완료!


In [ ]:
# [1] 필수 라이브러리 설치 (최초 실행 시 필요)
!pip install plotly

# [2] 데이터 분석 및 시각화 도구 로드
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px  # 호버 기능용 추가
import matplotlib.font_manager as fm

# [3] 그래프 한글 깨짐 방지 및 스타일 설정 (윈도우 환경)
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False

### 4.1 가설 검증: '이벤트가 혼잡도를 높이는가?'

**<표 1> 이벤트 발생 시 승하차 인원 변화 가설 검증 결과**

In [21]:
# 4. 가설 검증 결과 표 출력
df_hypothesis = pd.DataFrame({
    '구분': ['증가', '감소', '전체'],
    '건수': ['629건', '108건', '737건'],
    '비율': ['85.3%', '14.7%', '100%']
})
df_hypothesis

,구분,건수,비율
0,증가,629건,85.3%
1,감소,108건,14.7%
2,전체,737건,100%


**<그림 1> 호선별 평소 대비 이벤트 발생 시 혼잡도 비교**

In [ ]:
import pandas as pd
import plotly.graph_objects as go

# 1. 데이터 설정
labels = ['4호선', '2호선', '5호선']
baseline = [57500, 27800, 7800]    # 평소(Baseline)
event_day = [58000, 41500, 12000]  # 이벤트 당일

# 2. 그래프 생성 (객체 지향 방식)
fig = go.Figure()

# 평소(Baseline) 막대 추가
fig.add_trace(go.Bar(
    x=labels,
    y=baseline,
    name='평소(Baseline)',
    marker_color='#632a7a',
    text=[f"{v:,}" for v in baseline], # 막대 위에 표시될 텍스트
    textposition='auto',
    hovertemplate="<b>%{x}</b><br>평소 인원: %{y:,}명<extra></extra>"
))

# 이벤트 당일 막대 추가
fig.add_trace(go.Bar(
    x=labels,
    y=event_day,
    name='이벤트 당일',
    marker_color='#e27474',
    text=[f"{v:,}" for v in event_day],
    textposition='auto',
    hovertemplate="<b>%{x}</b><br>이벤트 당일: %{y:,}명<extra></extra>"
))

# 3. 레이아웃 및 한글 폰트 설정
fig.update_layout(
    title=dict(
        text='호선별 평소 대비 이벤트 발생 시 혼잡도 비교',
        x=0.5,
        font=dict(size=20)
    ),
    xaxis_title='지하철 호선',
    yaxis_title='평균 승하차 인원 합계 (명)',
    barmode='group', # 막대를 나란히 배치
    font=dict(
        family="Malgun Gothic", # 윈도우 한글 폰트
        size=12
    ),
    legend=dict(
        orientation="h", # 범례를 가로로
        yanchor="bottom",
        y=1.02,
        xanchor="right",
        x=1
    ),
    margin=dict(l=50, r=50, t=100, b=50),
    plot_bgcolor='white' # 배경을 깔끔하게 흰색으로
)

# 그리드 선 추가 (가독성용)
fig.update_yaxes(showgrid=True, gridwidth=1, gridcolor='LightGray')

# 4. 출력
fig.show()

**[결론]** 이벤트 발생 시 85.3%의 확률로 혼잡도가 증가하므로 가설은 타당함이 입증됨.

### 4.2 호선별 민감도 분석

**<표 2> 주요 호선별 이벤트 발생 건수 및 평균 상승률**

In [7]:
df_sensitivity = pd.DataFrame({
    '순위': [1, 2, 3],
    '호선': ['5호선', '2호선', '4호선'],
    '평균 상승률': ['68.44%', '56.84%', '0.10%'],
    '이벤트 수': ['403건', '258건', '76건']
})

df_sensitivity

,순위,호선,평균 상승률,이벤트 수
0,1,5호선,68.44%,403건
1,2,2호선,56.84%,258건
2,3,4호선,0.10%,76건


**<그림 2> 이벤트 카테고리별 승하차 상승률 분포**

In [ ]:
import pandas as pd
import plotly.express as px

# 1. 데이터 로드
file_path = r'C:\Users\Administrator\Desktop\새 폴더 (2)\subway-congestion-analysis\data\행사_시위\행사_시위_승하차인원_상승분석.csv'

try:
    df = pd.read_csv(file_path)
    print("✅ 데이터 로드 성공!")

    # 2. Plotly 박스플롯 생성
    fig = px.box(
        df, 
        x='카테고리', 
        y='상승률_%', 
        color='카테고리',       # 카테고리별 색상 자동 부여
        points='all',          # 모든 데이터 포인트를 옆에 표시 (stripplot 역할)
        title='이벤트 카테고리별 승하차 상승률 분포',
        labels={'상승률_%': '평소 대비 상승률 (%)', '카테고리': '이벤트 유형'},
        color_discrete_sequence=px.colors.qualitative.Set3 # 기존 Set3 느낌 유지
    )

    # 3. 호버 정보 및 스타일 설정
    fig.update_traces(
        marker=dict(size=3, opacity=0.5), # 점 크기 및 투명도 조정
        hovertemplate="<b>유형:</b> %{x}<br><b>상승률:</b> %{y:.2f}%<extra></extra>"
    )

    fig.update_layout(
        title_x=0.5, # 제목 가운데 정렬
        xaxis_tickangle=30, # 축 라벨 회전
        font=dict(family="Malgun Gothic", size=12), # 윈도우 한글 폰트
        showlegend=False, # 범례가 이미 X축에 있으므로 숨김 (깔끔하게)
        plot_bgcolor='white', # 배경색 흰색
        yaxis=dict(showgrid=True, gridcolor='LightGray') # 그리드 추가
    )

    # 0% 기준선 추가 (Matplotlib의 axhline 역할)
    fig.add_hline(y=0, line_dash="dash", line_color="red", opacity=0.5)

    # 4. 출력
    fig.show()

    # (선택) HTML 파일로 저장
    # fig.write_html("category_impact_interactive.html")

except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
except Exception as e:
    print(f"❌ 에러 발생: {e}")

✅ 데이터 로드 성공!


**분석 인사이트:**

* 5호선이 평균 68.44%로 가장 높은 민감도를 보임 (올림픽공원 등 대형 행사장 위치)
* 2호선은 56.84%로 종합운동장 등 스포츠 행사 영향
* 4호선은 0.10%로 상대적으로 낮음 (시위 등 소규모 이벤트 위주)

💡 **[데이터 인사이트: 4호선의 역설]**

* 4호선은 절대 인원은 가장 많으나 상승률은 0.1%로 최저임.
* 이는 평상시 유동인구(Baseline)가 워낙 거대한 '상시 포화 노선'이기 때문이며, 따라서 4호선은 이벤트 대응보다 상시 안전 관리가 더 중요함.

### 4.3 카테고리별 영향력 비교

**<표 3> 카테고리별 평균 상승률**

In [9]:
df_category = pd.DataFrame({
    '카테고리': ['야구', '행사', '시위'],
    '평균 상승률': ['73.70%', '65.74%', '0.38%'],
    '이벤트 수': ['141건', '484건', '112건']
})

df_category

,카테고리,평균 상승률,이벤트 수
0,야구,73.70%,141건
1,행사,65.74%,484건
2,시위,0.38%,112건


**<그림 3> 주요 역별 월간 이벤트 혼잡도 추이**

In [ ]:
import pandas as pd
import plotly.express as px

# 1. 데이터 로드
file_path = r'C:\Users\Administrator\Desktop\새 폴더 (2)\subway-congestion-analysis\data\행사_시위\행사_시위_승하차인원_상승분석.csv'

try:
    df = pd.read_csv(file_path)
    df['날짜'] = pd.to_datetime(df['날짜'])
    print("✅ 데이터 로드 성공!")

    # 2. 데이터 전처리 (상위 10개 역 추출)
    top_stations = df.groupby('역명')['상승률_%'].max().nlargest(10).index
    df_heatmap = df[df['역명'].isin(top_stations)].copy()
    df_heatmap['연월'] = df_heatmap['날짜'].dt.strftime('%Y-%m')

    # 피벗 테이블 생성 (행: 역명, 열: 연월)
    pivot_df = df_heatmap.pivot_table(
        index='역명', 
        columns='연월', 
        values='상승률_%', 
        aggfunc='mean'
    ).fillna(0)

    # 3. Plotly 히트맵 생성
    fig = px.imshow(
        pivot_df,
        labels=dict(x="날짜 (연-월)", y="역 이름", color="평균 상승률 (%)"),
        x=pivot_df.columns,
        y=pivot_df.index,
        text_auto=".1f",           # 칸 안에 수치 표시
        aspect="auto",
        color_continuous_scale='YlOrRd', # 노랑-주황-빨강 스타일
        title='주요 역별 월간 이벤트 혼잡도 추이 (상승률 TOP 10 역)'
    )

    # 4. 호버 설정 및 스타일 수정
    fig.update_traces(
        hovertemplate="<b>역명:</b> %{y}<br><b>기간:</b> %{x}<br><b>평균 상승률:</b> %{z:.1f}%<extra></extra>"
    )

    fig.update_layout(
        title_x=0.5,               # 제목 가운데 정렬
        font=dict(family="Malgun Gothic", size=12), # 윈도우 한글 폰트
        xaxis_nticks=len(pivot_df.columns), # X축 모든 연월 표시
        plot_bgcolor='white'
    )

    # 5. 출력
    fig.show()

except FileNotFoundError:
    print(f"❌ 파일을 찾을 수 없습니다: {file_path}")
except Exception as e:
    print(f"❌ 에러 발생: {e}")

✅ 데이터 로드 성공!


### 4.4 상승률 TOP 5 사례 및 심층 해석

In [11]:
df_top5 = pd.DataFrame({
    '순위': [1, 2, 3, 4, 5],
    '날짜': ['2023-06-17', '2023-06-18', '2024-06-22', '2024-11-02', '2023-06-24'],
    '호선': ['2호선', '2호선', '5호선', '5호선', '2호선'],
    '역명': ['종합운동장', '종합운동장', '올림픽공원', '올림픽공원', '종합운동장'],
    '카테고리': ['행사', '행사', '행사', '행사', '행사'],
    '당일 인원': ['97,726', '92,045', '19,434', '18,396', '83,460'],
    '평균': ['23,690', '23,823', '5,179', '5,179', '23,690'],
    '상승률': ['312.5%', '286.4%', '275.2%', '255.2%', '252.3%']
})

df_top5

,순위,날짜,호선,역명,카테고리,당일 인원,평균,상승률
0,1,2023-06-17,2호선,종합운동장,행사,"97,726","23,690",312.5%
1,2,2023-06-18,2호선,종합운동장,행사,"92,045","23,823",286.4%
2,3,2024-06-22,5호선,올림픽공원,행사,"19,434","5,179",275.2%
3,4,2024-11-02,5호선,올림픽공원,행사,"18,396","5,179",255.2%
4,5,2023-06-24,2호선,종합운동장,행사,"83,460","23,690",252.3%


💡 **[심층 해석: 야구와 행사의 혼잡 특성 차이]**

* **야구**: 시즌 중 지속 발생하는 '주기적 반복 혼잡'의 주범
* **행사**: 야구 이상의 폭발적 인파를 유발하는 '이벤트성 유동인구 폭증'의 주범
* **시사점**: 개별 이벤트의 단기 위험도는 대형 행사가 훨씬 높으므로 집중 관리가 필요함.

## 5. 결론 및 제언

### 5.1 주요 결론

1. **가설 지지**: 이벤트 발생 시 혼잡도 증가 확률 85.3%
2. **민감 노선**: 5호선과 2호선이 이벤트 영향에 가장 민감함
3. **영향도**: 야구(73.7%) > 행사(65.7%) > 시위(0.4%) 순

### 5.2 정책 제언

* **교통 수단 증편**: 종합운동장, 올림픽공원역 등 대형 행사장 인근 임시 열차 증차 및 시내버스 배차 단축
* **셔틀버스 의무화**: 대형 콘서트/경기 시 주최측과 협력하여 주요 거점(강남, 잠실 등) 연결 전용 셔틀 운영 권고
* **선별적 대응**: 상승률이 미미한 시위보다는 '이벤트성 유동인구 폭증'을 일으키는 대형 행사에 인력 및 자원 집중 투입 필요
* **예측 시스템**: 이벤트 일정과 연계한 실시간 혼잡도 예측 및 모니터링 시스템 구축 권장